# 🖖🏼 Mindfulness AI (mai) builder    
A taxonomy generator for an explainable health AI.
A novel, Focused Language Model builder for generating an indexable, semantically tuned taxonomy of medical content.   
Prompted with a natural English input (statement or query), a content (repository) and (explainable taxonomy model) are emitted/generated from (PubMed) content and (UMLS) concepts.   
The subject of "how to practice mindfulness" is generated in this example.   
The resulting JSON data model is paginated for efficiency of import to an Algolia index.   
The resulting Mindfulness AI is implemented with Algolia search, (here url to Algolia demo UI). 

In [ ]:
%pip install -q ipywidgets

### query and collect from pubmed

### query and collect from UMLS

### normalize vectors and generate JSON

### generate SVG

In [4]:
import os 
os.system('python mmlrestclient.py https://ii.nlm.nih.gov/metamaplite/rest/annotate mmlinput.txt --output mmloutput.txt --docformat sldiwi --resultformat mmi')
print('done')
# import mmlrestclient as mml

# mml mmlrestclient.py https://ii.nlm.nih.gov/metamaplite/rest/annotate mmlinput.txt --output mmloutput.txt --docformat sldiwi --resultformat mmi

done


In [11]:
import pandas as pd
# from urllib.request import urlopen
# pd.read_json("sample.json",lines=True,orient='columns')
df_data = pd.read_json('data.json')

newdf = df_data.drop("qualified",axis='columns')

print(newdf)

df_data.head()

# from pypmed import apis 

# search_criteria = {'author_first_name': 'rachel', 
#                    'author_last_name': 'gottschalk', 
#                    'institution':  'university of pittsburgh at pittsburgh',
#                    'publication_year': 2022
#                    }

# api_response = apis.query_pubmed_api(search_criteria=search_criteria)


# https://realpython.com/urllib-request/
# from urllib.request import urlopen
# response = None
# try:
#     # json https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&term=stem+cells+AND+free+fulltext%5bfilter%5d&retmode=json
#     # https://www.ncbi.nlm.nih.gov/books/NBK25499/?utm_source=blog&utm_medium=referral&utm_campaign=pubmed-api&utm_term=post1&utm_content=20210929link4#chapter4.ESearch
#     # https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=%E2%80%9Dasthma%20treatment%E2%80%9D[Title:~3]
#     response = urlopen("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&term=stem+cells+AND+free+fulltext%5bfilter%5d&retmode=json")
# except Exception as ex:
#     print(ex)
# else:
#     body = response.read()
# finally:
#     if response is not None:
#         response.close()

# with urlopen("https://www.example.com") as response:
#     body = response.read()


  provenance                                                url  \
0     pubmed  https://pubmed.ncbi.nlm.nih.gov/28031068/?form...   
1     pubmed  https://pubmed.ncbi.nlm.nih.gov/35634214/?form...   
2     github                                             github   
3   nlm umls                                           nlm umls   

                                               title                bonus  
0  What defines mindfulness-based programs? The w...                  NaN  
1  Meditators' Non-academic Definition of Mindful...                  NaN  
2                                                NaN                  NaN  
3                                                NaN  {'name': 'library'}  


,provenance,url,title,qualified,bonus
0,pubmed,https://pubmed.ncbi.nlm.nih.gov/28031068/?form...,What defines mindfulness-based programs? The w...,True,NaN
1,pubmed,https://pubmed.ncbi.nlm.nih.gov/35634214/?form...,Meditators' Non-academic Definition of Mindful...,True,NaN
2,github,github,NaN,True,NaN
3,nlm umls,nlm umls,NaN,False,{'name': 'library'}



### notes ...


pyMeSHSim
https://pubmed.ncbi.nlm.nih.gov/32552728/
https://pymeshsim.readthedocs.io/en/latest/
https://github.com/luozhhub/pyMeSHSim <=latest


UMLS Semantic Types
https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/Docs/SemanticTypes_2018AB.txt


[[{Value1:1},{value2:2}],[{value3:3},{value4:4}],.....]   
<= for data like that use this =>   
value1 = df['column_name'][0][0].get(Value1)   


health data modeling   
novel, focused language modeling   

![weird](/weird.svg)

Documents to get

"What defines mindfulness-based programs? The warp and the weft"
https://pubmed.ncbi.nlm.nih.gov/28031068/?format=pubmed
https://pubmed.ncbi.nlm.nih.gov/28031068/

"Meditators' Non-academic Definition of Mindfulness"
https://pubmed.ncbi.nlm.nih.gov/35634214/?format=pubmed
https://pubmed.ncbi.nlm.nih.gov/35634214/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9127491/

UMLS to get

https://www.nlm.nih.gov/research/umls/META3_current_semantic_types.html
https://www.nlm.nih.gov/research/umls/new_users/online_learning/UMLST_001.html

python mmlrestclient.py https://ii-public2vm.nlm.nih.gov/metamaplite/rest/annotate mmlinput.txt --output mmloutput.txt

python mmlrestclient.py https://ii.nlm.nih.gov/metamaplite/rest/annotate mmlinput.txt --output mmloutput.txt --docformat sldiwi --resultformat mmi

